In [68]:
import pandas as pd

import protfasta
import requests
import re
import warnings
warnings.filterwarnings("ignore")
import glob
import collections


from Bio import AlignIO
import matplotlib.pyplot as plt
from Bio.Align import AlignInfo

from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import rich
from rich_msa import RichAlignment

import logomaker

#Import modules
import gzip, logomaker, matplotlib.pyplot as plt, numpy as np, os, pandas as pd, protfasta


import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import itertools


from scipy import stats

# Percent identity
Saving the percent identity of each position, relative to the lambert TFs sequence.

Step 0: Check how many of CC's genes already have the correct isoform.

In [32]:
# Loading in info about the ADs
cc_names = pd.read_excel("../data/gene_names_with_location_and_pmvs.xlsx")
cc_names = cc_names.rename(columns = {"gene" : "Gene", "start" : "Start", "end" : "End"})
known_ADs = pd.read_csv("../output/known_ADs_considering_isoforms_and_canonical.csv")
cc_names = pd.merge(known_ADs[["uniprotID", "Gene", "Start", "End"]], cc_names)
ensts = pd.read_csv("../data/SFARI_TFs_with_ENST_corrected.csv", index_col = 0)
cc_names = pd.merge(cc_names, ensts)
cc_names["ENST"] = cc_names["ENST"].str.split(".").str[0]
cc_names["len"] = cc_names["End"] - cc_names["Start"] + 1
cc_names = cc_names.rename(columns = {"Start" : "start", "End" : "end"})
cc_TFs = cc_names[["uniprotID", "Gene"]].drop_duplicates()
cc_TFs

,uniprotID,Gene
0,O94983,CAMTA2
2,P11308,ERG
4,Q13422,IKZF1
5,O14770,MEIS2
6,Q15788,NCOA1
8,O95096,NKX2-2
9,P43354,NR4A2
11,P32242,OTX1
12,Q02548,PAX5
13,P26367,PAX6


In [33]:
# Assuming everything in prot alignments folder is actually correct
correc_isoforms = glob.glob("../data/zoonomia_toga_mca/prot_alignments/*")
correc_isoform_genes = [_.split("/")[-1].split(".")[0] for _ in correc_isoforms]
for gene in cc_TFs["Gene"]:
    if gene not in correc_isoform_genes:
        print(gene)

ERG
PAX6


Plan: For ERG and PAX6, align the correct isoform's sequence to the current protein alignment. Use that to save the percent identity per position.

# Add fragment: Align Lambert TFs sequence to the zoonomia alignment

In [35]:
lambert_TFs = pd.read_csv("../output/lambert_TFs_10-21-24_with_DBD_coords.csv", index_col = 0)
lambert_TFs

,id,ProteinSeq,DBD_coords_merged
0,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,MEPEGRGSLFEDSDLLHAGNPKENDVTAVLLTPGSQELMIRDMAEA...,"[[221, 243], [249, 271], [277, 299], [305, 327..."
1,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,"[[114, 182], [262, 347]]"
2,sp|A0PJY2|FEZF1_HUMAN Fez family zinc finger p...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,"[[260, 282], [288, 310], [316, 338], [344, 366..."
3,sp|A1A519|F170A_HUMAN Protein FAM170A OS=Homo ...,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,"[[1, 330]]"
4,sp|A1YPR0|ZBT7C_HUMAN Zinc finger and BTB doma...,MANDIDELIGIPFPNHSSEVLCSLNEQRHDGLLCDVLLVVQEQEYR...,"[[364, 386], [392, 414], [420, 442], [448, 469]]"
...,...,...,...
1608,sp|Q9Y6Q9|NCOA3_HUMAN Nuclear receptor coactiv...,MSGLGENLDPLASDSRKRKLPCDTPGQGLTCSGEKRRREQESKYIE...,"[[31, 83]]"
1609,sp|Q9Y6R6|Z780B_HUMAN Zinc finger protein 780B...,MVHGSVTFRDVAIDFSQEEWECLQPDQRTLYRDVMLENYSHLISLG...,"[[165, 187], [193, 215], [221, 243], [249, 271..."
1610,sp|Q9Y6X0|SETBP_HUMAN SET-binding protein OS=H...,MESRETLSSSRQRGGESDFLPVSSAKPPAAPGCAGEPLLSTPGPGK...,"[[583, 596], [1015, 1027], [1450, 1462]]"
1611,sp|Q9Y6X8|ZHX2_HUMAN Zinc fingers and homeobox...,MASKRKSTTPCMVRTSQVVEQDVPEEVDRAKEKGIGTPQPDVAKDS...,"[[78, 101], [110, 133], [271, 317], [442, 496]..."


In [41]:
# ! mkdir ../data/zoonomia_toga_mca/prot_alignments_for_vis/
#! mkdir ../data/zoonomia_toga_mca/prot_seqs_for_vis/

In [39]:
full_prot_alignments = glob.glob("../data/zoonomia_toga_mca/prot_alignments/*")
full_prot_alignments[:2]

['../data/zoonomia_toga_mca/prot_alignments/TP63.fasta',
 '../data/zoonomia_toga_mca/prot_alignments/RARG.fasta']

In [49]:
for gene in cc_TFs["Gene"]:
    df = lambert_TFs[lambert_TFs["id"].str.contains(gene)]
    protfasta.write_fasta([("LambertSeq", df["ProteinSeq"].iloc[0])], "../data/zoonomia_toga_mca/prot_seqs_for_vis/" + gene)
    #display(df)

    if gene in correc_isoform_genes:
        command = "mafft --auto --quiet --add ../data/zoonomia_toga_mca/prot_seqs_for_vis/" + gene + " --reorder --thread -1 ../data/zoonomia_toga_mca/prot_alignments/" + gene + ".fasta > ../data/zoonomia_toga_mca/prot_alignments_for_vis/" + gene + ".fasta"
    else:
        command = "mafft --auto --quiet --add ../data/zoonomia_toga_mca/prot_seqs_for_vis/" + gene + " --reorder --thread -1 ../data/zoonomia_toga_mca/prot_alignments_wrong_isoforms/" + gene + ".fasta > ../data/zoonomia_toga_mca/prot_alignments_for_vis/" + gene + ".fasta"
     
    os.system(command)

# Save percent identities for each

In [50]:
aligned_with_lambert = glob.glob("../data/zoonomia_toga_mca/prot_alignments_for_vis/*")
aligned_with_lambert

['../data/zoonomia_toga_mca/prot_alignments_for_vis/PAX6.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/ERG.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/PAX5.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/IKZF1.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/OTX1.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/NCOA1.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/MEIS2.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/PITX1.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/NR4A2.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/NKX2-2.fasta',
 '../data/zoonomia_toga_mca/prot_alignments_for_vis/CAMTA2.fasta']

In [67]:
for gene in cc_TFs["Gene"]:
    print(gene)
    alignment = AlignIO.read("../data/zoonomia_toga_mca/prot_alignments_for_vis/" + gene + ".fasta", "fasta")
    percent_identities = []
    summary_align = AlignInfo.SummaryInfo(alignment)
    
    for record in alignment:
        if "REFERENCE" in record.id:
            reference_seq = record.seq

        if "LambertSeq" in record.id:
            lambert_TF_seq = record.seq
    
    percent_identities = []
    positions = []
    
    # Iterate over each position in the alignment where ref_AA is not a gap
    for i in range(alignment.get_alignment_length()):
        ref_AA = reference_seq[i]
        lam_AA = lambert_TF_seq[i]

        
        # if ref_AA == "X" or ref_AA == "-":
        #     print(gene)

        if lam_AA != "-":
            # Get the column (position) for all sequences
            column = alignment[:, i]
        
            # Calculating percent identity
            count = column.count(ref_AA)
            percent_identity = 100 * count / len(column)
        
            # AAs.append(ref_AA)
            percent_identities.append(percent_identity)
            positions.append(lam_AA)
        
    percent_identities_df = pd.DataFrame({"percent_identity" : percent_identities, "pos" : positions})
    percent_identities_df.to_csv("../data/zoonomia_toga_mca/prot_alignment_percent_identities_for_vis/" + gene)
    #display(percent_identities_df)


CAMTA2
ERG
IKZF1
MEIS2
NCOA1
NKX2-2
NR4A2
OTX1
PAX5
PAX6
PITX1


## A. Correct isoforms